In [1]:
# exploring data in my new database!

In [67]:
import pymysql as mdb
import json
import re
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
import seaborn as sns
from sklearn import datasets, linear_model


In [3]:
%matplotlib qt
#matplotlib inline

In [4]:
con = mdb.connect('localhost','root','hobbes','kickstarter');

In [5]:
with con:
    cur = con.cursor()
    cur.execute("SHOW DATABASES")
    
    rows = cur.fetchall()
    for row in rows:
        print row
    cur.close()

('information_schema',)
('kickscrape',)
('kickstarter',)
('mysql',)
('performance_schema',)
('test',)
('testdb',)
('world_innodb',)


In [133]:
with con:
    cur = con.cursor()
    cur.execute("SELECT COUNT(Id) FROM Scraped_pages")
    rows = cur.fetchall()
    for row in rows:
        print row[0]
    cur.close()

20320


In [8]:
with con:
    cur = con.cursor()
    cur.execute("DESCRIBE Project_info")
    rows = cur.fetchall()
    for row in rows:
        print row[0]
    cur.close()

Id
Creator_name
Creator_url
Currency
Project_id
Category_name
Category_slug
Goal
Pledged
Project_date
Country
Location_club
Location_name
Blurb
N_backers
Name
URL
Launch_time
End_time


In [20]:
# start by plotting pledged, goal

In [11]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Goal, Pledged, Launch_time, N_backers FROM Project_info WHERE Category_slug LIKE 'technology%'")
    output = cur.fetchall()
    
    tech_data = np.array(output)
    
    cur.close()

In [18]:
np.shape(tech_data)

(9351, 4)

In [21]:
print tech_data[0][0]

155000


In [27]:
tech_df = pd.DataFrame(tech_data,columns=['Goal', 'Pledged', 'Launch_time', 'Num_backers'])

In [28]:
tech_df.head()

,Goal,Pledged,Launch_time,Num_backers
0,155000,125435,1416427327,879
1,30000,1968,1416304420,7
2,15000,816,1416013064,44
3,45000,5746,1415811884,252
4,30000,24112,1416049141,59


In [33]:
g = sns.pairplot(tech_df.ix[:100,:])

In [36]:
plt.close('all')

In [35]:
g = sns.pairplot(tech_df)

In [37]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Goal, Pledged, Launch_time, N_backers, Category_slug\
    FROM Project_info WHERE Category_slug LIKE 'technology%'")
    output = cur.fetchall()
    
    tech_data = np.array(output)
    
    cur.close()

In [38]:
tech_cat_df = pd.DataFrame(tech_data,columns=['Goal', 'Pledged', 'Launch_time', 'Num_backers','Subcategory'])

In [41]:
np.unique(tech_cat_df['Subcategory'])

array(['technology', 'technology/3d printing', 'technology/apps',
       'technology/camera equipment', 'technology/diy electronics',
       'technology/fabrication tools', 'technology/flight',
       'technology/gadgets', 'technology/hardware',
       'technology/makerspaces', 'technology/robots',
       'technology/software', 'technology/sound',
       'technology/space exploration', 'technology/wearables',
       'technology/web'], dtype=object)

In [60]:
sns.set_style("whitegrid")

In [61]:
plt.close('all')

In [62]:
g = sns.pairplot(tech_df)

In [63]:
#I don't know why my hue isn't working.

In [64]:
saveas_path = '/Users/jamie/insight data/figures/'

In [65]:
plt.savefig(saveas_path+'tech kickstarter pairplot.png',\
                bbox_inches='tight',dpi=200)

In [68]:
# now make a linear model.

In [70]:
tech_df.head()

,Goal,Pledged,Launch_time,Num_backers
0,155000,125435,1416427327,879
1,30000,1968,1416304420,7
2,15000,816,1416013064,44
3,45000,5746,1415811884,252
4,30000,24112,1416049141,59


In [79]:
goal = tech_df['Goal'].values
pledged = tech_df['Pledged'].values

In [83]:
np.shape(tech_df['Goal'])

(9351,)

In [107]:
goal_arrays = np.empty((9351,1))


In [110]:
for i in range(9351):
    goal_arrays[i] = goal[i]

In [112]:
print goal_arrays

[[ 155000.]
 [  30000.]
 [  15000.]
 ..., 
 [  50000.]
 [   7500.]
 [ 550000.]]


In [129]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model
regr.fit(goal_arrays,pledged)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [130]:
# The coefficients
print('Coefficients: \n', regr.coef_)

('Coefficients: \n', array([ 0.00102505]))


In [132]:
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((regr.predict(goal_arrays) - goal_arrays) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.10f' % regr.score(goal_arrays, goal_arrays))

Residual sum of squares: 2735739376092.92
Variance score: 0.0002031742
